# Homework 3 solutions

## Homework: Vector Search
In this homework, we'll experiemnt with vector with and without Elasticsearch

It's possible that your answers won't match exactly. If it's the case, select the closest one.

Q1. Getting the embeddings model
First, we will get the embeddings model multi-qa-distilbert-cos-v1 from the Sentence Transformer library

from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)
Create the embedding for this user question:

user_question = "I just discovered the course. Can I still join it?"
What's the first value of the resulting vector?

-0.24
-0.04
0.07
0.27

In [1]:
! pip -q install sentence-transformers

In [2]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path = "multi-qa-distilbert-cos-v1")

c:\Users\Odiaka\miniconda3\envs\alexey\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\Odiaka\miniconda3\envs\alexey\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Odiaka\.cache\huggingface\hub\models--sentence-transformers--multi-qa-distilbert-cos-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.52k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
user_question = "I just discovered the course. Can I still join it?"
vecs = embedding_model.encode(user_question)

In [4]:
vecs[0]

0.07822266

Prepare the documents
Now we will create the embeddings for the documents.

Load the documents with ids that we prepared in the module:

import requests

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()
We will use only a subset of the questions - the questions for "machine-learning-zoomcamp". After filtering, you should have only 375 documents

Q2. Creating the embeddings
Now for each document, we will create an embedding for both question and answer fields.

We want to put all of them into a single matrix X:

Create a list embeddings
Iterate over each document
qa_text = f'{question} {text}'
compute the embedding for qa_text, append to embeddings
At the end, let X = np.array(embeddings) (import numpy as np)
What's the shape of X? (X.shape). Include the parantheses.

In [5]:
import requests

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [7]:
ml_zoomcamp_docs = [doc for doc in documents if doc['course'] == 'machine-learning-zoomcamp']
len(ml_zoomcamp_docs)

375

In [8]:
import numpy as np

vectors = []
for doc in ml_zoomcamp_docs:
    qa_text = f"{doc['question']} {doc['text']}"
    vector = embedding_model.encode(qa_text)
    vectors.append(vector)

In [9]:
X = np.array(vectors)

In [10]:
X.shape

(375, 768)

Q3. Search
We have the embeddings and the query vector. Now let's compute the cosine similarity between the vector from Q1 (let's call it v) and the matrix from Q2.

The vectors returned from the embedding model are already normalized (you can check it by computing a dot product of a vector with itself - it should return 1.0). This means that in order to compute the coside similarity, it's sufficient to multiply the matrix X by the vector v:

scores = X.dot(v)
What's the highest score in the results?

65.0
6.5
0.65
0.065

In [11]:
X[0].dot(X[0])

1.0

In [12]:
# Compute cosine similarity
v = vecs
scores = X.dot(v)

# Find the highest score
highest_score = np.max(scores)
print(highest_score)

0.6506573


In [13]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=ml_zoomcamp_docs, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

In [14]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [15]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [16]:
from tqdm.auto import tqdm

def hit_rate(relevance_total):
    cnt = 0
    for line in relevance_total:
        if True in line:
            cnt += 1
    return cnt / len(relevance_total)

relevance_total = []
for q in tqdm(ground_truth):
    doc_id = q['document']
    results = search_engine.search(embedding_model.encode(q['question']), num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

print(hit_rate(relevance_total))

  0%|          | 0/1830 [00:00<?, ?it/s]

0.9398907103825137


Q5. Indexing with Elasticsearch
Now let's index these documents with elasticsearch

Create the index with the same settings as in the module (but change the dimensions)
Index the embeddings (note: you've already computed them)
After indexing, let's perform the search of the same query from Q1.

What's the ID of the document with the highest score?

In [22]:
from elasticsearch import Elasticsearch, helpers

es_client = Elasticsearch('http://localhost:9200')

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            }
        }
    }
}

index_name = "course-questions"

# Delete index if it exists
es_client.indices.delete(index=index_name, ignore=[400, 404])
es_client.indices.create(index=index_name, body=index_settings)

C:\Users\Odiaka\AppData\Local\Temp\ipykernel_22700\2928949247.py:30: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es_client.indices.delete(index=index_name, ignore=[400, 404])


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [23]:
# Prepare documents for indexing
es_documents = []
for doc, vector in zip(ml_zoomcamp_docs, vectors):
    es_doc = doc.copy()
    es_doc['vector'] = vector.tolist()  # Ensure the embedding is converted to a list
    es_documents.append(es_doc)

In [24]:
es_documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872',
 'vector': [0.08805904537439346,
  0.01559352595359087,
  0.07925580441951752,
  0.02527582459151745,
  0.07557652145624161,
  -0.03905971348285675,
  -0.04138142988085747,
  0.025291837751865387,
  0.024324113503098488,
  0.0036258557811379433,
  -0.0072828661650419235,
  -0.03287513926625252,
  0.061295606195926666,
  -0.057110074907541275,
  0.011677495203912258,
  -0.017944220453500748,
  0.044920630753040314,
  -0.054160621017217636,


In [25]:
# Bulk index documents
helpers.bulk(es_client, [{"_index": index_name, "_source": doc} for doc in es_documents])

(375, [])

In [26]:
user_question = "I just discovered the course. Can I still join it?"
v_query = embedding_model.encode(user_question).tolist()

search_query = {
    "size": 1,
    "query": {
        "script_score": {
            "query": {
                "match_all": {}
            },
            "script": {
                "source": "cosineSimilarity(params.query_vector, 'vector') + 1.0",
                "params": {
                    "query_vector": v_query
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)
top_hit = response['hits']['hits'][0]['_source']
print("ID of the document with the highest score:", top_hit['id'])


ID of the document with the highest score: ee58a693


Q6. Hit-rate for Elasticsearch
The search engine we used in Q4 computed the similarity between the query and ALL the vectors in our database. Usually this is not practical, as we may have a lot of data.

Elasticsearch uses approximate techniques to make it faster.

Let's evaluate how worse the results are when we switch from exact search (as in Q4) to approximate search with Elastic.

What's hitrate for our dataset for Elastic?

0.93
0.73
0.53
0.33

In [27]:
def elastic_search(query, course):
    v_query = embedding_model.encode(query).tolist()
    search_query = {
        "size": 5,
        "query": {
            "script_score": {
                "query": {
                    "bool": {
                        "filter": {
                            "term": {"course": course}
                        }
                    }
                },
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'vector') + 1.0",
                    "params": {
                        "query_vector": v_query
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

relevance_total = []
for q in ground_truth:
    doc_id = q['document']
    results = elastic_search(query=q['question'], course=q['course'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

def hit_rate(relevance_total):
    cnt = 0
    for line in relevance_total:
        if True in line:
            cnt += 1
    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0
    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score += 1 / (rank + 1)
    return total_score / len(relevance_total)

hit_rate_result = hit_rate(relevance_total)
mrr_result = mrr(relevance_total)
print("Hit-rate:", hit_rate_result)
print("MRR:", mrr_result)

Hit-rate: 0.9398907103825137
MRR: 0.8516484517304189
